# Parametric statistics

The `statsmodels` and `scipy` packages provide functions for parametric statistics (i.e. distribution based). `scipy` provides many of the underlying statiscal methods, while `statsmodels` provides the ability to run regression models. `numpy` provides functions for manipulating multidimensional arrays and matrices.

## Import libraries

As usual, we import `pandas`. We will also import `numpy` as `np`, `statsmodels.api` as `sm`. Scipy is frequently not aliased with a shorter name, but this is a matter of personal preference—there is no accepted standard like there is with `pandas`, `numpy`, and `statsmodels`. We import `scipy.stats` with no alias. We also import `statsmodels.stats`, which contains some utility functions, with no alias.

We are also going to import `matplotlib` to make some diagnostic plots for our results.

In [ ]:
%config InlineBackend.figure_format = "retina"

import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.outliers_influence
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

## Reading data from Excel

Next, we need to read in the data. We are using data on residential property assessments and sales from the [Wake County Assessor](https://www.wakegov.com/departments-government/tax-administration/data-files-statistics-and-reports/real-estate-property-data-files). I have already filtered the data file to only properties that sold since 2020 to reduce the file size.

The data are in Excel format. We can read Excel data directly into Python using the `pd.read_excel` function.

In [ ]:
data = pd.read_excel("../data/recent_sfh_sales.xlsx")
data

## Learning more information about the dataset

This dataset is large, and we can't see the whole thing at once. One potentially helpful thing to do is to print out the names of all of the columns. The names can be accessed with `data.columns` or `data.columns.values` - the second form will not truncate the list if it is long.

In [ ]:
data.columns.values

## Are tax assessments accurate?

The data include information both on sale cost (`Total_sale_Price`) and assessed value (`Assesed_Building_Value` and `Assessed_Land_Value`). In theory, these should track fairly closely, but in practice they may not. Let's first make a scatter plot plotting sale price vs. assessed value. We will first need to create a new variable that contains total assessed value (land plus building).

In [ ]:
data["Assessed_Total_Value"] = data.Assessed_Building_Value + data.Assessed_Land_Value

### Exercise

Make a scatter plot with sale price on the x axis and building value on the y axis. Label the axes. Only display the portion of the chart with sale and assessed values below $2 million. Add a 45 degree line to your plot with

`plt.plot([0, 2_000_000], [0, 2_000_000])`

### Testing the relationship between assessed and sale value

It looks like sale prices tend to be higher than assessed values. We can compute the mean difference between the two using the `np.mean` function.

Many functions are available in both Pandas and Numpy, and can often be used interchangeably.

I also added assert statements to make sure there are no missing values before we proceed with the analysis.

In [ ]:
np.mean(data.Total_sale_Price - data.Assessed_Total_Value)

In [ ]:
assert not data.Total_sale_Price.isnull().any()
assert not data.Assessed_Total_Value.isnull().any()

#### Evaluating statistical significance

We see that the average home sold for $87,000 more than it was assessed for. We can check whether this is statistically significant with a paired-sample _t_-test. This is available in the `scipy.stats.ttest_rel` function (for a _t_-test of related samples; `ttest_ind` is also available for independent samples).

In [ ]:
scipy.stats.ttest_rel(data.Total_sale_Price, data.Assessed_Total_Value)

Scipy also has functionality for working with distributions, so if you need to construct your own test you can. Below, we do this with a paired-sample t-test. I wouldn't recommend creating your own code for tests that are available in Scipy, but this demonstrates how you would do it if you were using a test that was not available in Scipy (for instance, if you did not have data on the two values, but only on the difference between them, and thus could not use `ttest_rel`

In [ ]:
# first, compute the test statistic. This is the average difference, divided by the standard error.
# the standard error is the standard deviation of the differences divided by the square root of the sample size.
avg_diff = np.mean(data.Total_sale_Price - data.Assessed_Total_Value)
std_err = np.std(data.Total_sale_Price - data.Assessed_Total_Value) / np.sqrt(len(data))
test_stat = avg_diff / std_err
test_stat

In [ ]:
# now, we can use the t cumulative distribution function to find the p-value
# We use abs to ensure the test statistic is positive, compute the cumulative distribution function
# with the correct number of degrees of freedom, and subtract from 1 to get the tail
p_value = 1 - scipy.stats.t.cdf(np.abs(test_stat), len(data))

# we then multiply the p_value by 2 for a two-tailed test. The *= operator multiplies the left hand side by
# the right hand, and saves the result into the variable on the left hand side
p_value *= 2

p_value

## Regression models

A very common form of inferential statistics is the regression model. `statsmodels` provides facilities to estimate many types of regression model; here, we'll estimate a linear regression model of home price.

If you're not familiar with linear regression, it's just a way to estimate an equation that best expresses an outcome in terms of other variables.

We will create a regression of home price on year built, acreage, heated area, city, and utilities available. First, we need to put the data in the right format.


### Exercise

The `UTILITIES` column contains which utilities are available at the property. It is a code that contains "W" if water is available, "S" is if sewer is available, "E" if electricity is available, "G" if gas is available, and "ALL" if all are available. For the regression model, we want to parse this out into four variables `water`, `sewer`, `gas`, and `electric`, one for each utility. We will use the string accessors we've seen before.

#### Check your work

To make sure this worked, let's print out the proportion of houses with each utility available. Almost every house should have electric available, while less will have water, gas, and sewer. We would expect fewer houses to have sewer than water, as houses with sewer system connections will almost always also have city water, while many homes have city water but rely on septic systems.

In [ ]:
data.electric.mean()

In [ ]:
data.water.mean()

In [ ]:
data.gas.mean()

In [ ]:
data.sewer.mean()

### Extracting variables for the regression

Next, we want to extract all of the variables for the regression. We will first create a data frame that has just the variables we care about. I am leaving out electric since it is true for almost all of the properties. I am also removing properties that sold for less than $15,000 as these are probably bad data or otherwise unrepresentative.

In [ ]:
reg_data = data.loc[data.Total_sale_Price > 15_000, ["Total_sale_Price", "Year_Built", "Deeded_Acreage", "HEATED_AREA", "CITY",
                 "water", "gas", "sewer"]]

### Check for null/missing values

Next, we want to check to see if there are any missing values in our regression variables.

In [ ]:
reg_data.isnull().sum()

### Exercise

There are some null values in the `CITY` variable. Look at some of these records, and look their addresses up in Google Maps (note: the address is split across multiple columns). Do you have an idea why these might be missing?

### Replace missing values

We don't actuall want to drop these records. Instead, we'll replace the missing values with a value indicating why they are missing. (Fill in the value below. I didn't fill it in so I wouldn't give away the answer above!)

In [ ]:
data.loc[data.CITY.isnull(), "CITY"] = ""

## Running the regression

`statsmodels` regression interface is a little bit more low-level than other statistical software you may have used. In particular, it will not automatically create dummy variables for non-numeric columns, and does not automatically add a constant - we have to do both of those things manually. Fortunately, there are functions available to help us.

`pd.get_dummies` will convert columns into dummy variables. `sm.add_constant` adds a constant column.

First, we'll add the dummy variables. By default `get_dummies` will convert all object and category columns, but I prefer to manually specify the columns I want dummies for, to ensure I get what I am intending to. For most types of regression you need to drop one of the dummy variables. Pandas can do this automatically by include `drop_first=True` in your call to `get_dummies`, but then you don't have any control over which category is the base category. Here, we will make Raleigh the base category.

In [ ]:
reg_data = pd.get_dummies(reg_data, columns=["CITY"]).drop(columns=["CITY_RAL"])

Now, we are ready to run the regression. `sm.add_constant` will return a new data frame, but since the constant column is not interesting it's typical to just include the call to `sm.add_constant` when running the regression. We also drop the total sale price column from the right hand side of the regression.

Running a regression in `statsmodels` is a two-step process. First, we create the regression model, then we use the `.fit()` function to estimate it.

I've divided the dependent variable by 1000, to make the results easier to interpret.

In [ ]:
model = sm.OLS(
    reg_data.Total_sale_Price / 1000, # dependent variable, referred to as "endog" in statsmodels documentation
    sm.add_constant(reg_data.drop(columns="Total_sale_Price"))
)
model.fit()

Running the code above gave us a common error message when running a regression; `statsmodels` complained that our data had type "object", which is non-numeric and cannot be used in a regression model. We can look at the dtypes to see if we can figure out what is going on.

In [ ]:
sm.add_constant(reg_data.drop(columns="Total_sale_Price")).dtypes

These all are numeric types (`bool` is True/False, but could also be considered 0/1). The issue is that `statsmodels` needs all of the columns to be the _same_ type, because it uses `numpy` internally; numpy matrices have a single type. This greatly increases speed, at the cost of some flexibility. The error message gave us a hint - we can use `np.asarray(dataframe)` to see what type it is choosing for the data.

In [ ]:
np.asarray(sm.add_constant(reg_data.drop(columns="Total_sale_Price")))

Well, there's the problem - it's getting converted to a matrix of "object". We can explicitly tell it that we want everything converted to a real number (`float64`) when we run the regression.

In [ ]:
model = sm.OLS(
    reg_data.Total_sale_Price / 1000, # dependent variable, referred to as "endog" in statsmodels documentation
    sm.add_constant(reg_data.drop(columns="Total_sale_Price")).astype("float64")
)
fit = model.fit()

We didn't get any errors, so the model ran. We can use the `fit.summary()` method to see a regression table.

In [ ]:
fit.summary()

### Regression diagnostics

It's a good idea to run some regression diagnostics after a regression. Statsmodels is again a bit lower-level than many statistical packages; many of the standard regression diagnostics need to be implemented manually. We see a note about multicollinearity in the results above, so we may want to check Variance Inflation Factors.

The `statsmodels` variance inflation factor function checks one column at a time, and requires the column number and a data frame with all of the columns used in estimation. Typically, we want to look all VIFs for all columns at once. We can use a loop to do this.

Loops let you write some code that will be executed many times. They are often discouraged in Python, because looping over many values can be slow (for instance, looping over all the rows of a data frame). However, here we are only looping over a few dozen columns, so performance should not be an issue.

We are using a for loop here; for loops run the code within them once for each value in a list, array, or other object that supports _iteration_. Here, we are iterating over the column names of the data, and using the `enumerate` function to get the column numbers. The code within a for loop is anything that is indented below line with "for" on it.

In [ ]:
# first, create a variable with exactly the data used in regression
exog = sm.add_constant(reg_data.drop(columns="Total_sale_Price")).astype("float64")

for column_index, column_name in enumerate(exog.columns):
    vif = statsmodels.stats.outliers_influence.variance_inflation_factor(exog, column_index)
    print(column_name, vif)

### Residuals vs fitted plot

Another common plot is the residuals vs. fitted plot, which can inform you of heteroskedasticity. The residuals and fitted values are available within the fit object.

There are other regression plots as well, but the point of this class is not to be an introduction to regression - this is just to show you how you might construct some of these plots yourself.

In [ ]:
plt.scatter(fit.fittedvalues, fit.resid, s=0.1)
plt.xlabel("Fitted values")
plt.ylabel("Residual")

The characteristic shape suggests heteroskedasticity. We can address that by taking the logarithm of the dependent variable. We can use the `np.log` function to transform the variable.

We'll also transform the year built variable. We might expect that there is a nonlinear relationship between year built and value, so we'll add a squared term. To use a squared (or other transformed) term in statsmodels, we just add the transformed column to the data frame. Interactions would be similar - create a new column multiplying existing columns together.

`**` is the power operator in Python.

In [ ]:
reg_data["Year_Built_Squared"] = reg_data.Year_Built ** 2

Now that there is another column in the `reg_data` data frame, we can just re-run the same regression code to include the new column. We use `np.log` to transform the sale price before running the regression.

In [ ]:
model = sm.OLS(
    np.log(reg_data.Total_sale_Price / 1000), # dependent variable, referred to as "endog" in statsmodels documentation
    sm.add_constant(reg_data.drop(columns="Total_sale_Price")).astype("float64")
)
fit = model.fit()
fit.summary()